In [2]:
%%bash
set -e

#### Install requirements
cd examples/t4rec_paper_experiments
pip install -r requirements.txt

### Get data
cd t4r_paper_repro

FEATURE_SCHEMA_PATH=../datasets_configs/ecom_rees46/rees46_schema.pbtxt
pip install gdown
gdown https://drive.google.com/uc?id=1NCFZ5ya3zyxPsrmupEoc9UEm4sslAddV
apt-get update -y
apt-get install unzip -y
DATA_PATH=/transformers4rec/examples/t4rec_paper_experiments/t4r_paper_repro/
unzip -d $DATA_PATH "rees46_ecom_dataset_small_for_ci.zip"
gdown https://drive.google.com/uc?id=18EllaKaodqaesrNJ3YGEmv0YUD3NX0vK
MODEL_PATH=/transformers4rec/TF4Rec/models/
mkdir -p /transformers4rec/TF4Rec/models/
unzip -d $MODEL_PATH "model.zip"
exit 0

Downloading...
From: https://drive.google.com/uc?id=1NCFZ5ya3zyxPsrmupEoc9UEm4sslAddV
To: /workspace/examples/t4rec_paper_experiments/t4r_paper_repro/rees46_ecom_dataset_small_for_ci.zip
100%|██████████| 43.4M/43.4M [00:07<00:00, 6.15MB/s]


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
unzip is already the newest version (6.0-25ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 74 not upgraded.
Archive:  rees46_ecom_dataset_small_for_ci.zip
   creating: /transformers4rec/examples/t4rec_paper_experiments/t4r_paper_repro/0001/
  inflating: /transformers4rec/examples/t4rec_paper_experiments/t4r_paper_repro/0001/valid.parquet  
 extracting: /transformers4rec/examples/t4rec_paper_experiments/t4r_paper_repro/0001/.zip  
  inflating: /transformers4rec/examples/t4rec_paper_experiments/t4r_paper_repro/0001/train.parquet  
  infla

Downloading...
From: https://drive.google.com/uc?id=18EllaKaodqaesrNJ3YGEmv0YUD3NX0vK
To: /workspace/examples/t4rec_paper_experiments/t4r_paper_repro/model.zip


Archive:  model.zip
   creating: /transformers4rec/TF4Rec/models/t4r_pytorch_pt/
  inflating: /transformers4rec/TF4Rec/models/t4r_pytorch_pt/config_GPU.pbtxt  
   creating: /transformers4rec/TF4Rec/models/t4r_pytorch_pt/1/
  inflating: /transformers4rec/TF4Rec/models/t4r_pytorch_pt/1/model.pkl  
  inflating: /transformers4rec/TF4Rec/models/t4r_pytorch_pt/1/model_info.json  
  inflating: /transformers4rec/TF4Rec/models/t4r_pytorch_pt/1/model.py  
  inflating: /transformers4rec/TF4Rec/models/t4r_pytorch_pt/1/model.pth  
   creating: /transformers4rec/TF4Rec/models/t4r_pytorch_pt/1/__pycache__/
  inflating: /transformers4rec/TF4Rec/models/t4r_pytorch_pt/1/__pycache__/model.cpython-38.pyc  
  inflating: /transformers4rec/TF4Rec/models/t4r_pytorch_pt/config.pbtxt  
  inflating: /transformers4rec/TF4Rec/models/t4r_pytorch_pt/config_CPU.pbtxt  
  inflating: /transformers4rec/TF4Rec/models/t4r_pytorch_pt/config.pbtxt.old  


Task exception was never retrieved
future: <Task finished name='Task-8' coro=<ScriptMagics.shebang.<locals>._handle_stream() done, defined at /usr/local/lib/python3.8/dist-packages/IPython/core/magics/script.py:211> exception=ValueError('Separator is not found, and chunk exceed the limit')>
Traceback (most recent call last):
  File "/usr/lib/python3.8/asyncio/streams.py", line 540, in readline
    line = await self.readuntil(sep)
  File "/usr/lib/python3.8/asyncio/streams.py", line 618, in readuntil
    raise exceptions.LimitOverrunError(
asyncio.exceptions.LimitOverrunError: Separator is not found, and chunk exceed the limit

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/magics/script.py", line 213, in _handle_stream
    line = (await stream.readline()).decode("utf8")
  File "/usr/lib/python3.8/asyncio/streams.py", line 549, in readline
    raise ValueError(e.args[0])


In [3]:
import glob
import logging
import os
from functools import partial
import pandas as pd
import cudf
import numpy as np
import nvtabular.inference.triton as nvt_triton
import tritonclient.grpc as grpcclient
import subprocess
import time

In [4]:
os.chdir('/workspace/examples/t4rec_paper_experiments/t4r_paper_repro')

In [5]:
eval_path = os.path.join(
    '/transformers4rec/examples/t4rec_paper_experiments/t4r_paper_repro/',
    str(2,).zfill(4),
    "valid.parquet",
)


In [6]:
# tritonserver --model-repository=/workspace/TF4Rec/models/

In [7]:
subprocess.Popen(['tritonserver',  '--model-repository=/transformers4rec/TF4Rec/models/'])

In [8]:
time.sleep(10)

I0215 04:33:37.500518 1321 pinned_memory_manager.cc:240] Pinned memory pool is created at '0x7f0666000000' with size 268435456
I0215 04:33:37.500846 1321 cuda_memory_manager.cc:105] CUDA memory pool is created on device 0 with size 67108864
I0215 04:33:37.502759 1321 model_lifecycle.cc:459] loading: t4r_pytorch_pt:1
I0215 04:33:41.177972 1321 python_be.cc:1856] TRITONBACKEND_ModelInstanceInitialize: t4r_pytorch_pt_0 (CPU device 0)
I0215 04:33:45.490882 1321 model_lifecycle.cc:694] successfully loaded 't4r_pytorch_pt' version 1
I0215 04:33:45.490943 1321 server.cc:563] 
+------------------+------+
| Repository Agent | Path |
+------------------+------+
+------------------+------+

I0215 04:33:45.490983 1321 server.cc:590] 
+---------+-------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Backend | Path                        

In [9]:
import tritonhttpclient
try:
    triton_client = tritonhttpclient.InferenceServerClient(url="localhost:8000", verbose=True)
    print("client created.")
except Exception as e:
    print("channel creation failed: " + str(e))
triton_client.is_server_live()

client created.
GET /v2/health/live, headers None
<HTTPSocketPoolResponse status=200 headers={'content-length': '0', 'content-type': 'text/plain'}>


/usr/local/lib/python3.8/dist-packages/tritonhttpclient/__init__.py:31: DeprecationWarning: The package `tritonhttpclient` is deprecated and will be removed in a future version. Please use instead `tritonclient.http`
  warnings.warn(


True

In [10]:
prediction_data = cudf.read_parquet(eval_path)

In [11]:
col_names = ['sess_pid_seq']
inputs = nvt_triton.convert_df_to_triton_input(col_names, prediction_data.loc[6, col_names], grpcclient.InferInput)

In [12]:
%%timeit

output_names = ["output"]

outputs = []
for col in output_names:
    outputs.append(grpcclient.InferRequestedOutput(col))
    
MODEL_NAME_PT = "t4r_pytorch_pt"
payload = cudf.DataFrame(data={'sess_pid_seq': np.random.randint(0, 390001, 20), 'id': 0}).groupby('id').agg({'sess_pid_seq': list})

with grpcclient.InferenceServerClient("localhost:8001") as client:
    col_names = ['sess_pid_seq']
    inputs = nvt_triton.convert_df_to_triton_input(col_names, payload, grpcclient.InferInput)
    response = client.infer(MODEL_NAME_PT, inputs)

6.09 ms ± 221 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
